In [26]:
%env CUDA_VISIBLE_DEVICES=0
#coding:utf-8

env: CUDA_VISIBLE_DEVICES=0


In [27]:
# from densenet.densenet_lib import Densenet_keras
import os 
import sys
sys.path.append(os.path.dirname(os.getcwd())+'densenet/')
import tensorflow as tf
import keras.backend as K
# import keys_union
from glob import glob
from PIL import Image
import time
import matplotlib.pyplot as plt
import difflib
import re
import random
import cv2
import densenet.keys as keys
import densenet.keys_keras as keys_keras
import densenet.densenet_lib as densenet_lib
import numpy as np
import densenet.keys_keras as keys_keras
import densenet.model as model
from densenet.model import densenet_cnn_model,densenet_rnn_model


In [28]:
tf.__version__

'1.9.0'

In [69]:
import imp
imp.reload(densenet_lib)
imp.reload(model)

<module 'densenet.model' from '/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/densenet/model.py'>

In [70]:
model_path = './models/trans_model/model_75layers_labeled_ch_len25_v1.pb'
characters = keys_keras.alphabet_union[:]
characters = characters[1:]+u'卍'
# len(characters)
densenet_tf = densenet_lib.Densenet_tf_multi_gpus(pb_model_path=model_path,characters=characters,fixed_batch_size=300)

In [ ]:
model_path = './models/75layers_labeled_ch_len25_v1-3.060-0.505.h5'
characters = keys_keras.alphabet_union[:]
characters = characters[1:]+u'卍'
# len(characters)
densenet_keras = densenet_lib.Densenet_keras(get_model_function=model.densenet_cnn_model,
                                             model_path=model_path,characters=characters)

In [ ]:
model_path = './densenet/models/weights_densenet.h5'
characters = keys.alphabet[:]
characters = characters[1:]+u'卍'
densenet_keras_5990 = densenet_lib.Densenet_keras(get_model_function=model.densenet_cnn_model,
                                                  model_path=model_path,characters=characters)

In [ ]:
model_path = './train/models/freezen_5layers_dataset_v1-0.672-0.894.h5'
characters = keys_keras.alphabet_union[:]
characters = characters[1:]+u'卍'
densenet_keras_2 = densenet_lib.Densenet_keras(get_model_function=model.densenet_cnn_model,
                                                model_path=model_path,characters=characters)

In [ ]:
# densenet_keras_5990.load_weights('./densenet/models/weights_densenet_with_name.h5')
densenet_keras_5990.basemodel = basemodel5
# densenet_keras_2.load_weights('./train/models/random_len20-06-0.400-0.926.h5')

# imgs_path = glob('/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/num_dataset_v2/train/*.png')
imgs_path = glob('/mnt/wuwenhui/git_ocr_project/keras_crnn/train/data/manual_crop/zh/*png')
img_path = random.choice(imgs_path)
# print(img_path)
# img = Image.open(img_path)
img = cv2.imread(img_path)
blank = np.ones((img.shape[0],50,3),dtype=np.uint8)*255
img = np.concatenate((img,blank),axis=1)
print(img.shape)
model = densenet_keras_5990
# img = cv2.resize(img,(800,32))
%time result = model.recognize(img)
# print(model.basemodel.summary())
plt.imshow(img)
print(result)


In [ ]:
# merge two model predict

# densenet_keras.load_weights('./train/models/75layers_labeled_ch_len25_v1-3.060-0.505.h5')
# densenet_keras_2.load_weights('./train/models/freezen_5layers_dataset_v1-0.672-0.894.h5')
    
# imgs_path = glob('/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/num_dataset_v2/train/*.png')
imgs_path = glob('/mnt/wuwenhui/git_ocr_project/keras_crnn/train/data/临时切割图/*png')
img_path = random.choice(imgs_path)
# print(img_path)
# img = Image.open(img_path)
img = cv2.imread(img_path)
blank = np.ones((img.shape[0],20,3),dtype=np.uint8)*255
img = np.concatenate((blank,img,blank),axis=1)
model1 = densenet_keras
model2 = densenet_keras_2
# img = cv2.resize(img,(800,32))
%time y_pred1 = model1.predict(img)
%time y_pred2 = model2.predict(img)
y_pred1 = y_pred1[:, :, :]
y_pred2 = y_pred2[:, :, :]

result = model1.decode((y_pred1+y_pred2)/2,with_confidence=True)
result2 = model2.recognize_with_confidence(img)
result3 = densenet_keras.recognize_with_confidence(img)
# print(model.basemodel.summary())
plt.imshow(img)
print(result)
print(result2,'model2')
print(result3,'model1')

In [72]:
img_paths = glob(u'./train/images/test_images20181122/*png')
# img_paths = glob(u'/mnt/wuwenhui/git_ocr_project/keras_crnn/train/data/manual_crop/zh/*png')
img_paths = img_paths*5
img_paths = random.sample(img_paths,300)
img_list = [ cv2.imread(img_path) for img_path in img_paths ]
result_list=[]
%time result_list = densenet_tf.recognize_on_batch(img_list,with_confidence=True,fixed_size=True)
print(len(result_list))
result_list

process use  0.5180456638336182
300
y_pred shape  (300, 100, 6043)


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 725 ms, sys: 357 ms, total: 1.08 s
Wall time: 1.04 s
300


[('3.00', 0.86272097),
 ('1.0', 0.95404613),
 ('0.00%', 0.8274964),
 ('6', 0.92408574),
 ('0.00', 0.87857306),
 ('价', 1.0),
 ('甲血氧饱和度监测', 0.99830526),
 ('3', 0.80649596),
 ('甲粪便隐血试验(OB）', 0.7757062),
 ('0.00%', 0.95893174),
 ('1.0', 0.9974058),
 ('', nan),
 ('21', 0.8727503),
 ('30.00', 0.99137354),
 ('0.00', 0.95672315),
 ('价', 0.9999367),
 ('H血清低密度脂蛋白胆固醇测定', 0.7767266),
 ('甲静脉置管冲洗', 0.99999195),
 ('3.00', 0.9491677),
 ('甲微量泵或输液泵加收', 0.92316246),
 ('82.88', 0.99228257),
 ('100.00', 0.94041413),
 ('甲钾测定（急诊）', 0.9171054),
 ('60.00', 0.78894407),
 ('9.00', 0.83435464),
 ('乙那屈肝素钙（速碧林）针4100单位*1支(4100单位*1', 0.9101561),
 ('50.01', 0.96862173),
 ('4.00', 0.8920442),
 ('1.0', 0.5068055),
 ('0.00', 0.97474813),
 ('28.60', 0.8749919),
 ('H血清低密度脂蛋白胆固醇测定', 0.7767266),
 ('1.00', 0.9762864),
 ('91.00', 0.58282554),
 ('金额', 0.9995663),
 ('甲静脉置管冲洗', 0.99999195),
 ('1.0', 0.5068055),
 ('0.00', 0.88458544),
 ('1.0', 0.95404613),
 ('10.00', 0.94601154),
 ('', nan),
 ('.00', 0.8802244),
 ('0.0%', 0.608184

In [13]:
{'2,1000':'3.2,4.1','2,300':'1.1,1.37',
 '1 1000':'3.5,4.5s','1,300':'1.16,1.44s',
 '4,1000':'2.9,4.4','4,300':'0.98,1.58'}

{'1 1000': '3.5,4.5s',
 '1,300': '1.16,1.44s',
 '2,1000': '3.2,4.1',
 '2,300': '1.1,1.37',
 '4,1000': '2.9,4.4',
 '4,300': '0.98,1.58'}

In [ ]:
img_paths

In [15]:
img_paths = glob('/mnt/wuwenhui/git_ocr_project/keras_crnn/train/data/manual_crop/zh/*png')
img_paths = img_paths*10
img_paths = random.sample(img_paths,300)
img_list = [ cv2.imread(img_path) for img_path in img_paths ]
result_list = []
start = time.time()
for img in img_list:
    result = densenet_tf.recognize_with_confidence(img)
    result_list.append(result)
print(time.time()-start)
print(len(result_list))
result_list

4.260918378829956
300


[('△△以上：住院床位费※', 0.92422354),
 ('乙型肝炎e抗体测定(AEY-HBe）※', 0.9056342),
 ('☆☆总※', 0.7788007),
 ('血清肌钙蛋白I测定(定量、干免疫学方法或', 0.93608665),
 ('梅毒螺旋体痔异抗体测定(其他免疫学万法', 0.9562152),
 ('C病房床位费※日', 0.97449064),
 ('▲(乙10%)艾速平※(乙10)艾司奥美拉唑钠※10m', 0.8415642),
 ('阴性症状评定量表SAS）爽', 0.92966753),
 ('尿液分析※', 0.99997234),
 ('j△△以上床位费※', 0.9352095),
 ('抗精神病药物治疗监测※', 0.99999046),
 ('梅毒螺旋体痔异抗体测定(其他免疫学万法', 0.9562152),
 ('!动静脉置管护理※口W', 0.8389641),
 ('乙型肝炎e抗体测定(AEY-HBe）※', 0.9056342),
 ('病房床位※口三', 0.7072904),
 ('j△△以上床位费※', 0.9352095),
 ('*△△以上：其他费※', 0.6745201),
 ('C病房床位费※日S', 0.732527),
 ('(甲)0.9%氯化钠(浙江济民)※(甲)0.9%氯化钠(塑袋', 0.87878317),
 ('(甲)0.%氯化钠(回音必）※(甲)0.9氯化钠(塑袋〉', 0.7952495),
 ('无机磷测定※', 0.9999984),
 ('肌酐测定※', 0.9999997),
 ('(甲)0.9%氯化钠(浙江济民)※(甲)0.9%氯化钠(塑袋', 0.87878317),
 ('!静脉输液※次', 0.95633954),
 ('△△以上：床位费※', 0.94156265),
 ('!智迅创源一次性使用避光式输液器(带针）※荷', 0.9159796),
 ('智迅衍液次性使用避光式输液器（带针）※付', 0.870545),
 ('赉便常痹※', 0.35259566),
 ('血清垂体泌乳素测定※', 0.9999881),
 ('血清丙氨酸氨基转移酶测定※', 0.96957344),
 ('△△以上：住院床位费※', 0.92422354),
 ('▲(

### 使用标注的样本测试模型的效果

In [ ]:
# with open('./train/create_dataset/total_medicine_v1.txt','w') as f:
#     for item in total_medicine:
#         f.write(item+'\n')

In [ ]:
import json
save_dir = './train/images/labeled_ch_data/valid/'
img_paths = glob(save_dir+'*png')

densenet_keras.load_weights('./train/models/75layers_labeled_ch_len25_v1-3.060-0.505.h5')

model = densenet_keras
error_list = []
# load json 文件
with open(save_dir+"valid_label.json",'r',encoding='utf-8') as json_file:
    label=json.load(json_file)
sum = 0
start = time



In [ ]:
%%time 
for img_path in img_paths:
    img = cv2.imread(img_path)
    img_name = os.path.split(img_path)[-1]
    result,confidence = model.recognize_with_confidence(img)
    if label[img_name] == result:
        sum +=1
    else:
        error_list.append((result,label[img_name]))
print(sum/len(img_paths))

In [ ]:
def check_name(check_str):
    len_str = len(check_str)
    cutoff = 0.7
    if len_str <= 3:
        cutoff = 0.5
    elif len_str <= 5:
        cutoff = 0.6
    return cutoff
with open('./train/create_dataset/name_dictionary_v1.2.txt') as f:
    medicine = f.readlines()
medicine = [name.split('\n')[0] for name in medicine ]
medicine = list(set(medicine))
len(medicine)
# medicine = [name.replace(' ','') for name in medicine]


def get_closest_matches(item):
    result,label = item
    cutoff = check_name(result)
    match_result = difflib.get_close_matches(result, medicine,cutoff=cutoff)
    if match_result:
        return (match_result,label)
    else:
        return (result,label)
    

In [ ]:
from multiprocessing import Pool,cpu_count
pool = Pool(cpu_count()//4)
# % time match_results = [get_closest_matches(item) for item in error_list ]

    
%time match_results = pool.map(get_closest_matches,error_list)
match_error = []
for results,label in match_results:
    if label == results[0]:
        sum +=1
    else:
        match_error.append((results[0],label))


In [ ]:
sum/len(img_paths)

### 尝试验证模型冻结层的效果

In [ ]:
from  densenet  import densenet
from keras.layers import Input,Dense
from keras.models import Model
characters = keys.alphabet[:]
characters = characters[:] 
nclass = len(characters)
print(len(characters))
input = Input(shape=(32, None, 1), name='the_input')
y_pred = densenet.dense_cnn(input, nclass)
basemodel = Model(inputs=input, outputs=y_pred)
modelPath = './densenet/models/weights_densenet_with_name.h5'

if os.path.exists(modelPath):
    print("Loading model weights...")
    basemodel.load_weights(modelPath)
    print('done!') 
  
    
# basemodel.save_weights('./models/no_use_model.h5')

In [ ]:
# characters = keys.alphabet_union[:]
# characters = characters[:] 
# nclass = len(characters)

basemodel2 = Model(inputs=input,outputs=basemodel.get_layer('flatten').output)
for i,layer in enumerate(basemodel2.layers):
    layer.set_weights = basemodel.layers[i].weights
flatten = basemodel.get_layer('flatten').output
y_pred = Dense(nclass, name='out', activation='softmax')(flatten)
basemodel3 =  Model(inputs=input,outputs=y_pred)
for i,layer in enumerate(basemodel3.layers):
    layer.set_weights = basemodel.layers[i].weights
# basemodel4 = Model(inputs=input,outputs=basemodel.get_layer('Time').output)
# basemodel4.load_weights()
# basemodel.summary()

In [ ]:
basemodel4 = Model(inputs=input,outputs=basemodel.get_layer('flatten').output)
basemodel4.load_weights('./models/pre_y_pred_weights.h5')
for i,layer in enumerate(basemodel.layers[:-1]):
    layer.set_weights = basemodel4.layers[i].weights
#     print(layer.name)

In [ ]:
basemodel3.layers[-1].set_weights = basemodel.layers[-1].weights
print(basemodel.layers[-1].name)

In [ ]:
basemodel3.save_weights('./models/merge_weights_pretrain_3.h5')

In [ ]:
print(nclass)
input1 = Input(shape=(32, None, 1), name='the_input')
y_pred1 = densenet.dense_cnn(input1, nclass)
basemodel5 = Model(inputs=input1, outputs=y_pred1)

In [ ]:
basemodel.save_weights('./densenet/models/weights_densenet_with_name.h5')

In [ ]:
basemodel5.load_weights('./densenet/models/weights_densenet.h5')

In [ ]:
basemodel.get_layer('pool3_0_avgpool')

In [ ]:
for i,layer in enumerate(basemodel5.layers):
    print(i,layer.name,layer) 
#     print(layer)


### 使用plot_model 观测模型情况

In [ ]:
from keras.utils import plot_model
plot_model(densenet_keras.basemodel,to_file='./models/rename_model.png')
plt.figure(figsize=(40,200))
img = Image.open('./models/rename_model.png')
plt.imshow(img)

### 多GPU预测性能测试

In [ ]:
from keras.utils import  multi_gpu_model
with tf.device('/cpu:0'):
    base_model =densenet_cnn_model()
    base_model.load_weights('./models/75layers_labeled_ch_len25_v1-3.060-0.505.h5')
parallel_model = multi_gpu_model(base_model)

In [ ]:
import numpy as np
from keras.backend import ctc_decode

characters = keys_keras.alphabet_union[:]
characters = characters[1:]+u'卍'
nClass = len(characters)
def recognize_on_batch(img_list,basemodel):
#         start = time.time()
    max_w = int(max([32.0/img.shape[0]*img.shape[1] for img in img_list]))
#         print(max_w)
#         assert len(img_list) < 128
    img_batch = np.ones([len(img_list),32,max_w,1])*0.5
    for i,img in enumerate(img_list):
        img_L = img.astype(np.float32)
        if len(img_L.shape) ==3:
            img_L = img_L[:,:,0]*114/1000 + img_L[:,:,1]* 587/1000 + img_L[:,:,2]* 299/1000 
        scale = img_L.shape[0]*1.0/32.0
        w = int(img_L.shape[1]/scale)
        img = cv2.resize(img_L,(w,32),cv2.INTER_AREA)
        img = img/ 255.0 - 0.5
        img = img.reshape((32, w, 1))
        img_batch[i,:,:w,:] = img
    start = time.time()
    y_pred = basemodel.predict_on_batch(img_batch)
    end = time.time()
    print('process use ' ,end-start)
    out_list = []
    for i in range(y_pred.shape[0]):
        y_encode = np.array([y_pred[i,:,:]]) 
        out = decode(y_encode)  ##
        out_list.append(out)
    return out_list


def decode(pred,with_confidence=False):
    text = pred.argmax(axis=2)[0]
    length = len(text)
    char_list = []
    n = nClass-1
    for i in range(length):
        # 这里text[i] != n就是处理出来的分隔符，text[i - 1] == text[i]处理重复的字符
        if text[i] != n and (not (i > 0 and text[i - 1] == text[i])):
                char_list.append(characters[text[i]])

    if with_confidence:
        pred_max = pred.max(axis=2)[0]
        confidence_list = [] 
        for i in range(length):
            # 这里text[i] != n就是处理出来的分隔符，text[i - 1] == text[i]处理重复的字符
            if text[i] != n and (not (i > 0 and text[i - 1] == text[i])):    
                    confidence_list.append(pred_max[i])
        length_confidence = len(confidence_list)

        if length_confidence < 3:
            confidence = np.mean(confidence_list)
        else :
            sorted_confidence = np.sort(confidence_list)
            confidence = np.mean(sorted_confidence[:length_confidence//2])
        return u''.join(char_list),confidence
    return u''.join(char_list) 


In [ ]:
img_paths = glob(u'/mnt/wuwenhui/git_ocr_project/keras_crnn/train/data/manual_crop/zh/*png')
img_paths = img_paths*50
img_paths = random.sample(img_paths,1000)
img_list = [ cv2.imread(img_path) for img_path in img_paths ]
%time result_list = recognize_on_batch(img_list,parallel_model)
print(len(result_list))
result_list

In [ ]:
import keras.backend.tensorflow_backend as KTF
KTF._get_available_gpus()

TypeError: import_graph_def() missing 1 required positional argument: 'graph_def'

In [ ]:
graph.

In [ ]:
len(a)